IMPORT LIBRARIES

In [1]:
from sqlalchemy import create_engine, Column, String, Integer, CHAR, ForeignKey, insert
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
print("libraries imported")

libraries imported


In [2]:
Base = declarative_base()

engine = create_engine("sqlite:///sql-murder-mystery.db", connect_args={'timeout': 30})
Base.metadata.create_all(bind=engine)

Session = sessionmaker(bind=engine)
session = Session()

In [3]:
Base.metadata.tables

FacadeDict({})

CREATING OBJECTS OF ALL THE TABLES

In [4]:
class crime_scene_report(Base):
    __tablename__ = "crime_scene_report"

    date = Column("date", Integer)
    type = Column("type", String)
    desc = Column("description", String, primary_key=True)
    city = Column("city", String)

    # def __init__(self,date,type,desc,city):
    #     self.date=date
    #     self.type=type
    #     self.desc=desc
    #     self.city=city

    def __repr__(self):
        return f"({self.date} || {self.type} || {self.desc} || {self.city})"

        
class drivers_license(Base):
    __tablename__ = "drivers_license"

    id=Column("id",Integer, primary_key=True)
    age=Column("age", Integer)
    height=Column("height", Integer)
    eye_color=Column("eye_color", String)
    hair_color=Column("hair_color", String)
    gender=Column("gender", String)
    plate_number=Column("plate_number", String)
    car_make=Column("car_make", String)
    car_model=Column("car_model", String)

    def __repr__(self):
        return f"({self.id} || {self.age} || {self.height} || {self.eye_color} || {self.hair_color} || {self.gender} || {self.plate_number} || {self.car_make} || {self.car_model})"


class income(Base):
    __tablename__ = "income"

    ssn = Column("ssn", CHAR, primary_key=True)
    annual_income = Column("annual_income", Integer)

    def __repr__(self):
        return f"({self.ssn} || {self.annual_income})"


class person(Base):
    __tablename__ = "person"
    
    id = Column("id", Integer, primary_key=True)
    name = Column("name", String) 
    license_id = Column("license_id", Integer, ForeignKey(drivers_license.id))
    address_number = Column("address_number", Integer)
    address_street_name = Column("address_street_name", String)
    ssn = Column("ssn", Integer, ForeignKey(income.ssn))    

    def __repr__(self):
        return f"({self.id} || {self.name} || {self.license_id} || {self.address_number} || {self.address_street_name} || {self.ssn})"
    

class facebook_event_checkin(Base):
    __tablename__ = "facebook_event_checkin"

    person_id = Column("person_id", Integer, ForeignKey(person.id), primary_key=True)
    event_id = Column("event_id", Integer)
    event_name = Column("event_name", String)
    date =Column("date", Integer)

    def __repr__(self):
        return f"({self.person_id} || {self.event_id} || {self.event_name} || {self.date})"


class get_fit_now_member(Base):
    __tablename__ = "get_fit_now_member"

    id = Column("id", String, primary_key=True)
    person_id = Column("person_id", Integer, ForeignKey(person.id))
    name = Column("name", String)
    membership_start_date = Column("membership_start_date", Integer)
    membership_status = Column("membership_status", Integer)    

    def __repr__(self):
        return f"({self.id} || {self.person_id} || {self.name} || {self.membership_start_date} ||{self.membership_status})" 


class get_fit_now_check_in(Base):
    __tablename__ = "get_fit_now_check_in"
    
    membership_id=Column("membership_id", String, ForeignKey(get_fit_now_member.id), primary_key=True)
    check_in_date=Column("check_in_date", Integer)
    check_in_time=Column("check_in_time", Integer)
    check_out_time=Column("check_out_time", Integer)

    def __repr__(self):
        return f"({self.membership_id} || {self.check_in_date} || {self.check_in_time} || {self.check_out_time})"


class interview(Base):
    __tablename__ = "interview"

    person_id = Column("person_id", Integer, ForeignKey(person.id), primary_key=True)
    transcript = Column("transcript", String)

    def __init__(self,person_id,transcipt):
        self.person_id=person_id
        self.transcript=transcript

    def __repr__(self):
        return f"({self.person_id} || {self.transcript})"

class solution(Base):

    __tablename__ = "solution"
    
    user = Column("user",Integer,primary_key=True)
    value = Column("value", String)

    def __init__(self,user,value):
        self.user = user
        self.value = value

    def __repr__(self):
        return f"({self.user} || {self.value})"
                   

QUERYING TO FIND THE SOLUTION

In [5]:
results = session.query(crime_scene_report).all()
print(len(results))

780


In [6]:
search = session.query(crime_scene_report).filter(crime_scene_report.date == 20180115).filter(crime_scene_report.city == "SQL City").filter(crime_scene_report.type == "murder").all()
print(*search, sep = "\n")

(20180115 || murder || Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave". || SQL City)


In [7]:
search1 = session.query(person).filter(person.name.like("Annabel%")).filter(person.address_street_name=="Franklin Ave").all()

print(*search1,sep ="\n")
# print(len(search1)) 

(16371 || Annabel Miller || 490173 || 103 || Franklin Ave || 318771143)


In [8]:
search2 = session.query(person).filter(person.address_street_name=="Northwestern Dr").order_by(person.address_number.desc()).first()
print(search2,sep="\n")

(14887 || Morty Schapiro || 118009 || 4919 || Northwestern Dr || 111564949)


In [9]:
search3 = session.query(interview).filter(interview.person_id.in_((16371,14887))).all()
print(*search3,sep="\n")

(14887 || I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".)
(16371 || I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.)


In [10]:
search4 = session.query(get_fit_now_member).filter(get_fit_now_member.membership_status.like("g%")).filter(get_fit_now_member.id.like("48Z%")).all()
print(*search4,sep="\n")
# print(search4)


(48Z7A || 28819 || Joe Germuska || 20160305 ||gold)
(48Z55 || 67318 || Jeremy Bowers || 20160101 ||gold)


In [11]:
search5 = session.query(person).filter(person.id.in_((28819,67318))).all()
print(*search5,sep="\n")

(28819 || Joe Germuska || 173289 || 111 || Fisk Rd || 138909730)
(67318 || Jeremy Bowers || 423327 || 530 || Washington Pl, Apt 3A || 871539279)


In [12]:
search6 = session.query(get_fit_now_check_in).filter(get_fit_now_check_in.check_in_date==20180109).all()
print(*search6,sep="\n")

(X0643 || 20180109 || 957 || 1164)
(UK1F2 || 20180109 || 344 || 518)
(XTE42 || 20180109 || 486 || 1124)
(1AE2H || 20180109 || 461 || 944)
(6LSTG || 20180109 || 399 || 515)
(7MWHJ || 20180109 || 273 || 885)
(GE5Q8 || 20180109 || 367 || 959)
(48Z7A || 20180109 || 1600 || 1730)
(48Z55 || 20180109 || 1530 || 1700)
(90081 || 20180109 || 1600 || 1700)


In [13]:
search6= session.query(drivers_license).filter(drivers_license.plate_number.like("H42W%")).all()
print(search6)

[(183779 || 21 || 65 || blue || blonde || female || H42W0X || Toyota || Prius)]


In [14]:
search7=session.query(person).filter(person.license_id==183779).all()
print(search7)

[(78193 || Maxine Whitely || 183779 || 110 || Fisk Rd || 137882671)]


In [15]:
search7 = session.query(interview).filter(interview.person_id.in_((28819,67318,78193))).all()
print(search7)

[(67318 || I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5" (65") or 5'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.
)]


In [16]:
# stmt = insert(solution).values(user=1,value="Jeremy Bowers")

# print(stmt)

# session.execute(
#     solution.insert().
#     values(user=1,value="Jeremy Bowers")
# )

ans = solution(1,"Jeremy Bowers")
print(ans)

(1 || Jeremy Bowers)


In [17]:
session.add(ans)
# session.commit()

In [18]:
search8 = session.query(solution).all()
print(search8)

[(0 || Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer.)]


In [19]:
search9 = session.query(drivers_license).filter(drivers_license.height.in_((65,67))).filter(drivers_license.hair_color.like("red")).filter(drivers_license.gender=="female").filter(drivers_license.car_make=="Tesla").all()
print(*search9,sep="\n")

(918773 || 48 || 65 || black || red || female || 917UU3 || Tesla || Model S)


In [20]:
search10 = session.query(person).filter(person.license_id == 918773).all()
print(search10)

[(78881 || Red Korb || 918773 || 107 || Camerata Dr || 961388910)]


In [21]:
search11 = session.query(income).filter(income.ssn == 961388910).all()
print(search11)

[(961388910 || 278000)]


In [22]:
ans1 = solution(1,"Red Korb")
session.add(ans1)

In [23]:
search12 = session.query(solution).all()
print(search12)

session.close()

[(0 || Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer.)]


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_16896\4065606535.py:1: SAWarning: New instance <solution at 0x1acd6aefb20> with identity key (<class '__main__.solution'>, (1,), None) conflicts with persistent instance <solution at 0x1acd6ad3a60>
  search12 = session.query(solution).all()
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_16896\4065606535.py:1: SAWarning: Identity map already had an identity for (<class '__main__.solution'>, (1,), None), replacing it with newly flushed object.   Are there load operations occurring inside of an event handler within the flush?
  search12 = session.query(solution).all()
